In [81]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
import faiss
import pickle

In [85]:
MovieDataSet = pd.read_csv("MovieFinal.csv")
MovieDataSet.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine,poster
0,Underworld: Blood Wars,Fantasy Action Thriller Horror,Anna Foerster,Kate Beckinsale,Theo James,Lara Pulver,Vampire death dealer Selene fends off brutal a...,Underworld: Blood Wars Fantasy Action Thriller...,https://image.tmdb.org/t/p/original/v1ciDCWMG4...
1,Arsenal,Thriller Crime,Steven C. Miller,Adrian Grenier,Johnathon Schaech,Nicolas Cage,After the deadbeat brother of a businessman is...,Arsenal Thriller Crime Steven C. Miller Adria...,https://image.tmdb.org/t/p/original/b9vwgGqWge...
2,Between Us,Drama,Rafael Palacio Illingworth,Olivia Thirlby,Ben Feldman,Adam Goldberg,A New York couple and their Midwestern friends...,Between Us Drama Rafael Palacio Illingworth O...,https://image.tmdb.org/t/p/original/uieZWH2BOz...
3,Monster Trucks,Action Comedy Science Fiction,Chris Wedge,Lucas Till,Jane Levy,Amy Ryan,Tripp is a high school senior with a knack for...,Monster Trucks Action Comedy Science Fiction C...,https://image.tmdb.org/t/p/original/jDKONUh6CD...
4,The Bye Bye Man,Horror Thriller,Stacy Title,Douglas Smith,Doug Jones,Michael Trucco,When three college students move into an old h...,The Bye Bye Man Horror Thriller Stacy Title D...,https://image.tmdb.org/t/p/original/bape74Eeli...


In [22]:
%%time
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(MovieDataSet["combine"])

CPU times: user 25min 40s, sys: 1min 22s, total: 27min 3s
Wall time: 3h 20min 11s


In [86]:
vectors_movie = encoder.encode(MovieDataSet["movie_title"])

In [87]:
vectors_movie.shape[1]

768

In [88]:
# creating an index equal to the length of the vector
index = faiss.IndexFlatL2(vectors.shape[1])
index_movie = faiss.IndexFlatL2(vectors_movie.shape[1])

In [90]:
#adding the created vector embeddings to the index (This is the vector database)
index.add(vectors)
index_movie.add(vectors_movie)

In [91]:
search_query = "Spiderman"
search_vec = encoder.encode(search_query)

In [61]:
search_query = "Creed the boxer is such a great fighter. he loves boxing in the ring. last time he knocked his opponent down"
search_vec = encoder.encode(search_query)


In [107]:
search_vec = search_vec.reshape((1,-1))
search_vec.shape

(1, 768)

In [108]:
distances, I = index_movie.search(search_vec, 10)
I

array([[2014, 1890, 1897, 1903, 1786,  497,  620, 1660,  123,  502]])

In [109]:
I = I.reshape(-1,)
I

array([2014, 1890, 1897, 1903, 1786,  497,  620, 1660,  123,  502])

In [110]:
MovieDataSet.iloc[I]

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine,poster
2014,Spider-Man,Action Adventure Fantasy Romance,Sam Raimi,J.K. Simmons,James Franco,Kirsten Dunst,"After reuniting with Gwen Stacy, Brooklyn’s fu...",Spider-Man Action Adventure Fantasy Romance S...,https://image.tmdb.org/t/p/original/8Vt6mWEReu...
1890,The Amazing Spider-Man,Action Adventure Fantasy,Marc Webb,Emma Stone,Andrew Garfield,Chris Zylka,Peter Parker is an outcast high schooler aband...,The Amazing Spider-Man Action Adventure Fanta...,https://image.tmdb.org/t/p/original/jIfkQNARYy...
1897,Spider-Man 2,Action Adventure Fantasy Romance,Sam Raimi,J.K. Simmons,James Franco,Kirsten Dunst,"For Peter Parker, life is busy. Between taking...",Spider-Man 2 Action Adventure Fantasy Romance...,https://image.tmdb.org/t/p/original/olxpyq9kJA...
1903,The Amazing Spider-Man 2,Action Adventure Fantasy Sci-Fi,Marc Webb,Emma Stone,Andrew Garfield,B.J. Novak,"For Peter Parker, life is busy. Between taking...",The Amazing Spider-Man 2 Action Adventure Fan...,https://image.tmdb.org/t/p/original/c3e9e18SSl...
1786,Spider-Man: Across the Spider-Verse,Action Adventure Animation Science Fiction,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,"After reuniting with Gwen Stacy, Brooklyn’s fu...",Spider-Man: Across the Spider-Verse Action Adv...,https://image.tmdb.org/t/p/original/8Vt6mWEReu...
497,Spider-Man: Into the Spider-Verse,Action Adventure Animation Science Fiction,"Bob Persichetti, Peter Ramsey, Rodney Rothman",Shameik Moore,Jake Johnson,Hailee Steinfeld,Struggling to find his place in the world whil...,Spider-Man: Into the Spider-Verse Action Adven...,https://image.tmdb.org/t/p/original/iiZZdoQBEY...
620,Spider-Man: Far From Home,Action Adventure Science Fiction,Jon Watts,Tom Holland,Samuel L. Jackson,Zendaya,Peter Parker and his friends go on a summer tr...,Spider-Man: Far From Home Action Adventure Sci...,https://image.tmdb.org/t/p/original/4q2NNj4S5d...
1660,Spider-Man: Across the Spider-Verse*,Action Adventure Animation Science Fiction,NaN,NaN,NaN,NaN,"After reuniting with Gwen Stacy, Brooklyn’s fu...",Spider-Man: Across the Spider-Verse* Action Ad...,https://image.tmdb.org/t/p/original/8Vt6mWEReu...
123,Spider-Man: Homecoming,Action Adventure Science Fiction Drama,Jon Watts,Tom Holland,Michael Keaton,Jon Favreau,Following the events of Captain America: Civil...,Spider-Man: Homecoming Action Adventure Scienc...,https://image.tmdb.org/t/p/original/c24sv2weTH...
502,Aquaman,Action Adventure Fantasy,James Wan,Jason Momoa,Amber Heard,Willem Dafoe,Once home to the most advanced civilization on...,Aquaman Action Adventure Fantasy James Wan Ja...,https://image.tmdb.org/t/p/original/uCg2HPY7rB...


In [111]:
movie_file_path = "MovieNameIndex.pk1"
with open(movie_file_path, "wb") as f:
    pickle.dump(vectors_movie, f)

In [83]:
file_path = "VectorOverviewIndex.pk1"
with open(file_path, "wb") as f:
    pickle.dump(vectors, f)